In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


In [11]:
data = pd.read_csv('Housing.csv')


X = data[['area', 'bedrooms', 'bathrooms']].values
y = data['price'].values


scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_normalized = scaler_X.fit_transform(X)
y_normalized = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

X_b_normalized = np.c_[np.ones((X_normalized.shape[0], 1)), X_normalized]


def hypothesis(X, w):
    return X @ w

def compute_loss(X, y, w):
    m = len(y)
    predictions = hypothesis(X, w)
    return (1 / (2 * m)) * np.sum((predictions - y) ** 2)

def gradient_step(X, y, w, learning_rate):
    m = len(y)
    predictions = hypothesis(X, w)
    gradient = (1 / m) * X.T @ (predictions - y)
    return w - learning_rate * gradient

def gradient_descent(X, y, learning_rate=0.01, epochs=1000):
    w = np.zeros(X.shape[1]) 
    for i in range(epochs):
        w = gradient_step(X, y, w, learning_rate)
        if np.isnan(w).any() or np.isinf(w).any():
            print("Произошло переполнение или NaN значение. Прекращаем обучение.")
            break
    return w

w_gradient_descent = gradient_descent(X_b_normalized, y_normalized, learning_rate=0.01, epochs=1000)

y_pred_gradient_descent_normalized = hypothesis(X_b_normalized, w_gradient_descent)

y_pred_gradient_descent = scaler_y.inverse_transform(y_pred_gradient_descent_normalized.reshape(-1, 1)).flatten()

lin_reg = LinearRegression()
lin_reg.fit(X, y)
y_pred_sklearn = lin_reg.predict(X)

mse_gradient_descent = mean_squared_error(y, y_pred_gradient_descent)
mse_sklearn = mean_squared_error(y, y_pred_sklearn)
print("Оптимальные веса (Градиентный спуск):", w_gradient_descent)
print("Оптимальные веса (Scikit-learn):", np.r_[lin_reg.intercept_, lin_reg.coef_])

print("MSE (Градиентный спуск):", mse_gradient_descent)
print("MSE (Scikit-learn):", mse_sklearn)

Оптимальные веса (Градиентный спуск): [3.04382025e-16 4.39444124e-01 1.60696001e-01 3.72187885e-01]
Оптимальные веса (Scikit-learn): [-1.73171608e+05  3.78762754e+02  4.06820034e+05  1.38604950e+06]
MSE (Градиентный спуск): 1791170165422.4973
MSE (Scikit-learn): 1791170049977.319
